In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv
import re

# Step 01

Pull information about all refrigerators, what their part numbers/manufactures part numbers are, a description of the part, and a link to the product

In [3]:
fridge_url = f"https://www.partselect.com/Fridge-Parts.htm" 
page = requests.get(fridge_url)

In [4]:
soup = BeautifulSoup(page.content, "html.parser")

In [5]:
popular_parts_section = soup.find('h2', class_='section-title', string='Popular Fridge Parts')
popular_parts_section

<h2 class="section-title bold mt-4 mb-3 mb-sm-4">Popular Fridge Parts</h2>

In [6]:
print(page.text)


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <meta name="theme-color" content="#337778" />
    <meta name="HandheldFriendly" content="True" />
    <meta name="MobileOptimized" content="320" />
    <meta name="format-detection" content="telephone=no" />
    <meta http-equiv="X-UA-Compatible" content="IE=EmulateIE8" />
    <link rel="apple-touch-icon-precomposed" href="https://partselectcom-gtcdcddbene3cpes.z01.azurefd.net/shortcut.png" />
    <link rel="apple-touch-icon" sizes="180x180" href="https://partselectcom-gtcdcddbene3cpes.z01.azurefd.net/apple-icon-180x180.png">
    <link rel="mask-icon" href="https://partselectcom-gtcdcddbene3cpes.z01.azurefd.net/safari-pinned-tab.svg" color="#ff9933">
    
    <link href="https://partselectcom-gtcdcddbene3cpes.z01.azurefd.net/favicon.ico" rel="icon" type="image/x-ic

In [7]:
price_parts_div = popular_parts_section.find_all_next('div', class_='nf__part__left-col__basic-info__stock')
for i, part in enumerate(price_parts_div):
    price = part.find('div',class_='mb-1 mb-sm-2 js-tooltip').get_text(strip=True)
    print(price)

In Stock
In Stock
In Stock
In Stock
In Stock
In Stock
In Stock
In Stock
In Stock
In Stock


In [8]:
image_titles = popular_parts_section.find_all_next('img', alt=True)
# for i, img in enumerate(image_titles):
#     print(i+1, ". ", img['alt'])

In [9]:
parts = popular_parts_section.find_all_next('div', class_='nf__part__detail')
for i,part in enumerate(parts):
        part_number = part.find('div', class_='nf__part__detail__part-number').strong.text
        manufacturer_number = part.find_all('div', class_='nf__part__detail__part-number')[1].strong.text
        explanation_div = part.find('div', class_='nf__part__detail__symptoms')
        explanation = explanation_div.previous_sibling.strip() if explanation_div else "No explanation available"

        # print(i+1, ". Part Select Number:", part_number)
        # print("Manufacturer Part Number:", manufacturer_number)
        # print("Explanation:", explanation)
        # print()

In [10]:
soup.find('div', class_='mt-sm-2 price').get_text(strip=True)

'$90.73'

In [11]:
image_titles = popular_parts_section.find_all_next('img', alt=True)
parts = popular_parts_section.find_all_next('div', class_='nf__part__detail')
price_parts_div = popular_parts_section.find_all_next('div', class_='nf__part__left-col__basic-info__price')
stock_parts_div = popular_parts_section.find_all_next('div', class_='nf__part__left-col__basic-info__stock')

# Check if the part number in the title matches the manufacturer number
pattern = r'Part Number: (\w+)'

for i, (img, part, prices, stocks) in enumerate(zip(image_titles, parts, price_parts_div, stock_parts_div), start=1):
    print(i, ". Image Title:", img['alt'])
    match = re.search(pattern, img['alt'])

    if match:
        part_number_from_title = match.group(1)
        part_number = part.find('div', class_='nf__part__detail__part-number').strong.text
        manufacturer_number = part.find_all('div', class_='nf__part__detail__part-number')[1].strong.text
        detail_title = part.find('a', class_='nf__part__detail__title').text.strip()
        price = prices.find('div',class_='mt-sm-2 price').get_text(strip=True)
        stock = stocks.find('div',class_='mb-1 mb-sm-2 js-tooltip').get_text(strip=True)
        if part_number_from_title == manufacturer_number:
            explanation_div = part.find('div', class_='nf__part__detail__symptoms')
            explanation = explanation_div.previous_sibling.strip() if explanation_div else "No explanation available"
            # Extracting URL
            url = part.find('a', class_='nf__part__detail__title')['href']
            print("    Part Select Number:", part_number)
            print("    Manufacturer Part Number:", manufacturer_number)
            print("    Manufacturer:", url.split('-')[1])
            print("    Explanation:", explanation)
            print("    Detail Title:", detail_title)
            print("    Link to product:", url)
            print("    Price:", price)
            print("    Current", stock)
            print() 

1 . Image Title: Refrigerator Ice and Water Filter – Part Number: EDR1RXD1
    Part Select Number: PS11701542
    Manufacturer Part Number: EDR1RXD1
    Manufacturer: Whirlpool
    Explanation: This is an OEM replacement ice and water filter for your refrigerator. In side-by-side refrigerators, it is found in the top right corner or the bottom left grille. In bottom-freezer refrigerators, the filter is in the bottom left grille. In top-freezer units, the filter is located in the middle of the refrigerator. This NSF certified filter reduces 28 contaminants including pharmaceuticals, metals, minerals, pesticides, cysts, industrial chemicals, Class I Particulates and chlorine. It is made up of coconut carbon block and has three layers of filtration. The first layer is particle filtration that helps block larger contaminants. The second layer is micro-filtration, which prevents medium-sized impurities from passing. The third layer is called absorption filtration, which absorbs microscopic-

# Step 02

Within each product, pull information about their troubleshooting, such as which products they can replace, and what symptoms they fix

In [12]:
product_url = "https://www.partselect.com/PS11701542-Whirlpool-EDR1RXD1-Refrigerator-Ice-and-Water-Filter.htm?SourceCode=18"
product_page = requests.get(product_url)
soup = BeautifulSoup(product_page.content, "html.parser")
troubleshooting_section = soup.find("div", {"id": "Troubleshooting"})

if troubleshooting_section:
    symptoms_fixed = troubleshooting_section.find_next("div", class_="bold", string="This part fixes the following symptoms:").next_sibling.strip()
    
    replacements = soup.find("div", class_="bold", string="Part# EDR1RXD1 replaces these:").find_next_sibling("div")
    if replacements:
        replacements_list = [r.strip() for r in replacements.get_text(strip=True).split(',')]

    product_info_div = soup.find("div", class_="bold mb-1", string="This part works with the following products:")
    product_info = product_info_div.find_next("div").text.strip()
        
    print("    Symptoms Fixed:", symptoms_fixed)
    print("    This product can replace:", replacements_list)
    print("    This product also works with:", product_info.replace("This part works with the following products:", "").strip())
else:
    print("Troubleshooting section not found.")


    Symptoms Fixed: Not dispensing water | Ice maker not making ice | Ice maker won’t dispense ice | Leaking
    This product can replace: ['AP5982535', '9900', '9981', 'FILTER 1', 'FILTER1', 'W10217316', 'W10291030', 'W10295370', 'W10295370A', 'W10569758', 'W10569760', 'W10569761', 'W10735398']
    This product also works with: Whirlpool, Maytag, KitchenAid, Kenmore.


In [13]:
# import requests
# from bs4 import BeautifulSoup

# product_url = "https://www.partselect.com/PS11701542-Whirlpool-EDR1RXD1-Refrigerator-Ice-and-Water-Filter.htm?SourceCode=18"
# product_page = requests.get(product_url)
# soup = BeautifulSoup(product_page.content, "html.parser")
# troubleshooting_section = soup.find("div", {"id": "Troubleshooting"})

# if troubleshooting_section:
#     symptoms_fixed = troubleshooting_section.find_next("div", class_="bold", string="This part fixes the following symptoms:").next_sibling.strip()
#     print("Symptoms Fixed:", symptoms_fixed)
    
#     # Inserted code to extract replacements
#     target_div = soup.find("div", class_="bold", string="Part# EDR1RXD1 replaces these:")
#     if target_div:
#         values_div = target_div.find_next_sibling("div")
#         if values_div:
#             replacements = [value.strip() for value in values_div.get_text(strip=True).split(',')]
#             print("Replacements:", replacements)
#     else:
#         print("Replacements information not found.")
# else:
#     print("Troubleshooting section not found.")


In [14]:
# product_url = "https://www.partselect.com/PS11701542-Whirlpool-EDR1RXD1-Refrigerator-Ice-and-Water-Filter.htm?SourceCode=18"
# product_page = requests.get(product_url)
# soup = BeautifulSoup(product_page.content, "html.parser")
# troubleshooting_section = soup.find("div", {"id": "Troubleshooting"})

# if troubleshooting_section:
#     symptoms_fixed = troubleshooting_section.find_next("div", class_="bold", string="This part fixes the following symptoms:").next_sibling.strip()
#     print("Symptoms Fixed:", symptoms_fixed)
    
#     # Extract the desired product information
#     product_info_div = soup.find("div", class_="bold mb-1", string="This part works with the following products:")
#     if product_info_div:
#         product_info = product_info_div.find_next("div").text.strip()
#         print("Product Information:", product_info)
#     else:
#         print("Product information not found.")
    
#     # Inserted code to extract replacements
#     target_div = soup.find("div", class_="bold", string="Part# EDR1RXD1 replaces these:")
#     if target_div:
#         values_div = target_div.find_next_sibling("div")
#         if values_div:
#             replacements = [value.strip() for value in values_div.get_text(strip=True).split(',')]
#             print("Replacements:", replacements)
#     else:
#         print("Replacements information not found.")
# else:
#     print("Troubleshooting section not found.")

# Step 03

Pull customer Reviews

In [15]:
product_url = "https://www.partselect.com/PS11701542-Whirlpool-EDR1RXD1-Refrigerator-Ice-and-Water-Filter.htm?SourceCode=18"
product_page = requests.get(product_url)
soup = BeautifulSoup(product_page.content, "html.parser")
average_product_rating = soup.find('div', class_='pd__cust-review__header__rating__chart--border').get_text(strip=True)
num_reviews = soup.find('span', class_='rating__count').get_text().strip()
average_customer_rating = soup.find('div', class_='pd__repair-story__raiting mt-2 mb-4').get_text(strip=True).replace("What's this?","").strip()
print("Average Rating:", average_product_rating, "based on", num_reviews)
print("Ease of repair rating:",average_customer_rating)

Average Rating: 4.8 based on 100 Reviews
Ease of repair rating: Average Repair Rating: 4.4 / 5.0, 15 reviews.


In [16]:
print(product_page.text)


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <meta name="theme-color" content="#337778" />
    <meta name="HandheldFriendly" content="True" />
    <meta name="MobileOptimized" content="320" />
    <meta name="format-detection" content="telephone=no" />
    <meta http-equiv="X-UA-Compatible" content="IE=EmulateIE8" />
    <link rel="apple-touch-icon-precomposed" href="https://partselectcom-gtcdcddbene3cpes.z01.azurefd.net/shortcut.png" />
    <link rel="apple-touch-icon" sizes="180x180" href="https://partselectcom-gtcdcddbene3cpes.z01.azurefd.net/apple-icon-180x180.png">
    <link rel="mask-icon" href="https://partselectcom-gtcdcddbene3cpes.z01.azurefd.net/safari-pinned-tab.svg" color="#ff9933">
    
    <link href="https://partselectcom-gtcdcddbene3cpes.z01.azurefd.net/favicon.ico" rel="icon" type="image/x-ic

# Step 04
Pull Questions and Answers

In [17]:
product_url = "https://www.partselect.com/PS11701542-Whirlpool-EDR1RXD1-Refrigerator-Ice-and-Water-Filter.htm?SourceCode=18"
product_page = requests.get(product_url)
soup = BeautifulSoup(product_page.content, "html.parser")
questions_and_answers = soup.find('div', id='QuestionsAndAnswersContent')
questions, answers, model_nos = [], [],[]

if questions_and_answers:
    question_elements = questions_and_answers.find_all('div', class_='qna__question')
    
    for question_element in question_elements:
        question = question_element.find(class_='js-searchKeys').text.strip()
        model_number_element = question_element.find(class_='bold mt-3 mb-3')
        if model_number_element:
            model_number = model_number_element.text.strip()
            model_number = model_number.replace("For model number","").strip()
        else:
            model_number = "Model number not found"
        answer = question_element.find(class_='qna__ps-answer__msg').text.strip()
        
        questions.append(question + "This is for model number " + model_number)
        answers.append(answer)
        model_nos.append(model_number)
        
        print("Question:", question)
        print("Answer:", answer)
        print("Model number:", model_number)
        print("-" * 50)
else:
    print("Questions and Answers section not found.")

Question: Your water filter w10295370a , what all does this filter out of the water that passes through it ?
Answer: Hi Greg, Thank you for the question. The filter reduces the most Contaminants based on NSF rated 6 month refrigerator filters flowing .Reduces 24 contaminants including: pharmaceuticals, metals, minerals, pesticides, cysts, industrial chemicals, Class I Particulates and Chlorine. Hope this helps!
Model number: Model number not found
--------------------------------------------------
Question: I have this Whirlpool and can not find where water filter is.
Answer: Hi Ken,

Thank you for your inquiry. For your model, the filter is hidden behind a small panel on the left hand side of your kickplate at the bottom of your refrigerator. You should be able to just open that panel and replace your filter. Good luck with your repair.
Model number: WRS325FDAM04
--------------------------------------------------
Question: The description says the filter is on the left bottom behind a

In [18]:
questions

['Your water filter w10295370a , what all does this filter out of the water that passes through it ?This is for model number Model number not found',
 'I have this Whirlpool and can not find where water filter is.This is for model number WRS325FDAM04',
 'The description says the filter is on the left bottom behind a kick plate; however, mine is located on the top right  that is in the refrigerator section. Is this the correct part?This is for model number MSS26C6MFW00',
 "To replace the water filter, the small door on the left side of the kick plate should open.  Does it open from the right or the left?  It is hard to open and I'm afraid to force it as a previous question said they broke it off and it can't be easily fixed.This is for model number KSC24C8EYB02",
 'How do I repair the spring loaded door where the water filter goes. The spring came off and it is supposed to pull out the water filter when you open the little door with the spring.This is for model number WRS325FDAB02',
 'H

In [19]:
answers

['Hi Greg, Thank you for the question. The filter reduces the most Contaminants based on NSF rated 6 month refrigerator filters flowing .Reduces 24 contaminants including: pharmaceuticals, metals, minerals, pesticides, cysts, industrial chemicals, Class I Particulates and Chlorine. Hope this helps!',
 'Hi Ken,\n\nThank you for your inquiry. For your model, the filter is hidden behind a small panel on the left hand side of your kickplate at the bottom of your refrigerator. You should be able to just open that panel and replace your filter. Good luck with your repair.',
 'Hello Judy,\n\nThanks for your question. Yes, this is the correct part for your model - its location can vary from model to model.\n\nI hope this helps.',
 'Hi Susan, Thank you for the question. Looks like your door opens from the bottom. Hope this helps!',
 'Hi Betty,\n\nThank you for the question. The only way to repair the filter door would be to replace the filter housing. Unfortunately they do not sell the door par

# Step 06

Info on most popular models

In [2]:
popular_fridge_url = "https://www.partselect.com/Refrigerator-Parts.htm"
popular_models = requests.get(popular_fridge_url)
soup = BeautifulSoup(popular_models.content, "html.parser")
part_section = soup.find('h2', id='ShopByPartType')
if part_section:
    parts_list = part_section.find_next('ul', class_='nf__links')
    if parts_list:
        parts = [part.text.replace(' ', '-') for part in parts_list.find_all('a')]
        parts_with_links = ["https://www.partselect.com/{}.htm".format(part.replace(' ', '-')) for part in parts]
        print(parts_with_links)


['https://www.partselect.com/Refrigerator-Bearings.htm', 'https://www.partselect.com/Refrigerator-Blades.htm', 'https://www.partselect.com/Refrigerator-Brackets-and-Flanges.htm', 'https://www.partselect.com/Refrigerator-Caps-and-Lids.htm', 'https://www.partselect.com/Refrigerator-Circuit-Boards-and-Touch-Pads.htm', 'https://www.partselect.com/Refrigerator-Compressors.htm', 'https://www.partselect.com/Refrigerator-Deflectors-and-Chutes.htm', 'https://www.partselect.com/Refrigerator-Dispensers.htm', 'https://www.partselect.com/Refrigerator-Door-Shelves.htm', 'https://www.partselect.com/Refrigerator-Doors.htm', 'https://www.partselect.com/Refrigerator-Drawers-and-Glides.htm', 'https://www.partselect.com/Refrigerator-Drip-Bowls.htm', 'https://www.partselect.com/Refrigerator-Ducts-and-Vents.htm', 'https://www.partselect.com/Refrigerator-Electronics.htm', 'https://www.partselect.com/Refrigerator-Elements-and-Burners.htm', 'https://www.partselect.com/Refrigerator-Fans-and-Blowers.htm', 'https

In [9]:
fridge_parts = pd.DataFrame(parts_with_links, columns=['Fridge_replacements'])
fridge_parts.to_csv("/Users/sarah_prakriti_peters/Documents/Instalily/data/fridge_replacements.csv")
# fridge_parts

In [10]:
popular_dishwasher_url = "https://www.partselect.com/Dishwasher-Parts.htm"
popular_models = requests.get(popular_dishwasher_url)
soup = BeautifulSoup(popular_models.content, "html.parser")
part_section = soup.find('h2', id='ShopByPartType')
if part_section:
    parts_list = part_section.find_next('ul', class_='nf__links')
    if parts_list:
        parts = [part.text.replace(' ', '-') for part in parts_list.find_all('a')]
        parts_with_links = ["https://www.partselect.com/{}.htm".format(part.replace(' ', '-')) for part in parts]
        print(parts_with_links)

['https://www.partselect.com/Dishwasher-Bearings.htm', 'https://www.partselect.com/Dishwasher-Brackets-and-Flanges.htm', 'https://www.partselect.com/Dishwasher-Caps-and-Lids.htm', 'https://www.partselect.com/Dishwasher-Circuit-Boards-and-Touch-Pads.htm', 'https://www.partselect.com/Dishwasher-Dishracks.htm', 'https://www.partselect.com/Dishwasher-Dispensers.htm', 'https://www.partselect.com/Dishwasher-Doors.htm', 'https://www.partselect.com/Dishwasher-Drawers-and-Glides.htm', 'https://www.partselect.com/Dishwasher-Ducts-and-Vents.htm', 'https://www.partselect.com/Dishwasher-Elements-and-Burners.htm', 'https://www.partselect.com/Dishwasher-Filters.htm', 'https://www.partselect.com/Dishwasher-Grilles-and-Kickplates.htm', 'https://www.partselect.com/Dishwasher-Handles.htm', 'https://www.partselect.com/Dishwasher-Hardware.htm', 'https://www.partselect.com/Dishwasher-Hinges.htm', 'https://www.partselect.com/Dishwasher-Hoses-and-Tubes.htm', 'https://www.partselect.com/Dishwasher-Insulation.h

In [11]:
dishwasher_parts = pd.DataFrame(parts_with_links, columns=['Dishwasher_replacements'])
dishwasher_parts.to_csv("/Users/sarah_prakriti_peters/Documents/Instalily/data/dishwasher_replacements.csv")
# fridge_parts